#simulation d'un bot et analyse statistique

On considère un batiment constitué de 6 pièces agencées sous la forme  
A B C  
D E F  
Il peut d'agir d'un bâtiment réel ou virtuel (par ex dans un jeu vidéo).  
Un bot représentant un personnage non joueur mène la garde dans le bâtiment.  
Au temps 0, il se trouve en A.
A chaque nouveau pas de temps, le bot se déplace au hasard dans une pièce adjacente, avec la même probabilité pour chacune des directions possibles. Le bot ne se déplace pas en diagonale.
Lorsque le bot revient *pour la troisième fois* dans la pièce F, alors son mouvement cesse et il reste
définitivement en F.

On souhaiterait répondre aux questions suivantes :  
1) Quel est le temps moyen que prend le processus pour que le bot termine son mouvement ?  
2) Quand le mouvement du bot est terminé, combien de fois le bot est-il passé en D en moyenne ? (un joueur se cache en D alors cette quantité est importante)

## version hyper basique sans fonctions sans dict sans rien

In [1]:
import random

In [2]:

t = 0
en_cours = True
nb_visites_F = 0
position = 'A'
while(en_cours):
    print(position, end = ' ')
    if position=='A':
        r = random.randint(1,2)
        if r==1:
            position = 'B'
        else:
            position = 'D'
    elif position=='B':
        r = random.randint(1,3)
        if r==1:
            position = 'A'
        elif r==2:
            position = 'E'
        else:
            position = 'C'
    elif position=='C':
        r = random.randint(1,2)
        if r==1:
            position = 'B'
        else:
            position = 'F'
    elif position=='D':
        r = random.randint(1,2)
        if r==1:
            position = 'A'
        else:
            position = 'E'
    elif position=='E':
        r = random.randint(1,3)
        if r==1:
            position = 'D'
        elif r==2:
            position = 'B'
        else:
            position = 'F'
    elif position=='F':
        nb_visites_F += 1
        if nb_visites_F < 3:
            r = random.randint(1,2)
            if r==1:
                position = 'E'
            else:
                position = 'C'
        else:
            en_cours = False


A D A D A D A D E D A B E F C F E B C B A D E B E D E B A B E F 

Le code précédent fait juste une simulation. Pour répondre aux deux questions précédentes on peut le répéter et collecter des infos statistiques.

In [4]:
nb_repet = 100
somme_durees_totale = 0
somme_passages_en_D = 0
for i in range(nb_repet):
    t = 0
    en_cours = True
    nb_visites_F = 0
    position = 'A'

    duree_totale = 0
    passages_en_D = 0
    while(en_cours):
        duree_totale += 1
        if position=='A':
            r = random.randint(1,2)
            if r==1:
                position = 'B'
            else:
                position = 'D'
        elif position=='B':
            r = random.randint(1,3)
            if r==1:
                position = 'A'
            elif r==2:
                position = 'E'
            else:
                position = 'C'
        elif position=='C':
            r = random.randint(1,2)
            if r==1:
                position = 'B'
            else:
                position = 'F'
        elif position=='D':
            passages_en_D += 1
            r = random.randint(1,2)
            if r==1:
                position = 'A'
            else:
                position = 'E'
        elif position=='E':
            r = random.randint(1,3)
            if r==1:
                position = 'D'
            elif r==2:
                position = 'B'
            else:
                position = 'F'
        elif position=='F':
            nb_visites_F += 1
            if nb_visites_F < 3:
                r = random.randint(1,2)
                if r==1:
                    position = 'E'
                else:
                    position = 'C'
            else:
                en_cours = False
    somme_durees_totale += duree_totale
    somme_passages_en_D += passages_en_D
print("duree moyenne d'un épisode :", somme_durees_totale/nb_repet)
print("nb moyen de passages en D :", somme_passages_en_D/nb_repet)

duree moyenne d'un épisode : 24.18
nb moyen de passages en D : 4.13


## Version plus avancee avec des dictionnaires pour les voisins et des fonctions

In [5]:
voisins = {'A' : ['B', 'D'], 
           'B' : ['A', 'E', 'C'], 
           'C': ['B','F'], 
           'D' : ['A', 'E'], 
           'E' : ['D', 'B', 'F'], 
           'F': ['E','C'],}

In [6]:
def transition(piece):
    r = random.randint(0,len(voisins[piece])-1)
    return voisins[piece][r]

In [7]:
def episode():
    en_cours = True
    position = 'A'
    historique = ['A']
    nb_visites_F = 0
    while en_cours:
        position = transition(position)
        historique.append(position)
        if position=='F':
            nb_visites_F += 1
            if nb_visites_F==3:
                en_cours = False
    return historique

In [8]:
def moyenne(l):
    return sum(l) / len(l)

In [9]:
nb_repet = 1000
simulations = [episode() for i in range(nb_repet)]

In [7]:
duree_moyenne = moyenne([len(e) for e in simulations])
nb_moyen_passage_D = moyenne([e.count('D') for e in simulations])

In [8]:
print(duree_moyenne, nb_moyen_passage_D)

24.306 3.896
